# Text 1: Vector space models
**Internet Analytics - Lab 4**

---

**Group:** *Y*

**Names:**

* *Kristian Aurlien*
* *Mateusz Paluchowski*

---

#### Instructions

*This is a template for part 1 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [1]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from utils import load_json, load_pkl

courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')

In [2]:
# Custom python packages we are going to use:

#  nltk - Robust NLP library
from nltk.tokenize import wordpunct_tokenize 
from nltk.stem.porter import *

from collections import OrderedDict
from operator import itemgetter
import operator
from collections import Counter
import re

## Exercise 4.1: Pre-processing

#### Check provided data

In [3]:
len(courses)

2562

In [4]:
#Let's check if the stopwords in our dict are unique.
len(stopwords) == len(set(stopwords))

True

In [5]:
stop_words = set(stopwords)

In [6]:
# However courses are not unique:
courses[1006]['courseId'] == courses[80]['courseId']

True

In [7]:
unique_courses = list({v['courseId']:v for v in courses}.values())

In [8]:
len(unique_courses)

854

#### Remove the stopwords and punctuation. Get the word frequency count.

In [9]:
# Punctuation
stop_words.update(['.', ',', '-', '/', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '&','%', '.,', '),', ').', '://', '&)', '=', '",', ');', '):', '%).'])

In [10]:
# Dict for word frequency count. (should it be per corpus or per document?)
words_used_freq = {}
processed_courses = unique_courses.copy()

for id, course in enumerate(unique_courses):
    description = course['description']
    
    list_of_words = []
    for i in wordpunct_tokenize(description):
        i_low = i.lower() #convert to lowercase
        if i_low not in stop_words: #remove stopwords & punctuation
            list_of_words += [i_low]
            if i_low in words_used_freq:
                words_used_freq[i_low] += 1
            else:
                words_used_freq[i_low] = 1
    processed_courses[id]['tokens'] = list_of_words

In [11]:
processed_courses[0]['tokens']

['latest',
 'developments',
 'processing',
 'generations',
 'organic',
 'composites',
 'discussed',
 'nanocomposites',
 'adaptive',
 'composites',
 'biocomposites',
 'presented',
 'product',
 'development',
 'cost',
 'analysis',
 'study',
 'markets',
 'practiced',
 'team',
 'work',
 'content',
 'basics',
 'composite',
 'materialsconstituentsprocessing',
 'compositesdesign',
 'composite',
 'structures',
 'current',
 'developmentnanocomposites',
 'textile',
 'compositesbiocompositesadaptive',
 'composites',
 'applicationsdriving',
 'forces',
 'marketscost',
 'analysisaerospaceautomotivesport',
 'keywords',
 'composites',
 'applications',
 'nanocomposites',
 'biocomposites',
 'adaptive',
 'composites',
 'design',
 'cost',
 'learning',
 'prerequisites',
 'required',
 'courses',
 'notion',
 'polymers',
 'recommended',
 'courses',
 'polymer',
 'composites',
 'learning',
 'outcomes',
 'end',
 'student',
 'propose',
 'suitable',
 'design',
 'production',
 'performance',
 'criteria',
 'producti

In [12]:
words_used_freq_ordered = OrderedDict(sorted(words_used_freq.items(),key = operator.itemgetter(1),reverse = True))
words_used_freq_ordered

OrderedDict([('methods', 1627),
             ('learning', 1477),
             ('student', 1212),
             ('content', 908),
             ('students', 840),
             ('design', 781),
             ('courses', 760),
             ('systems', 750),
             ('analysis', 719),
             ('end', 675),
             ('assessment', 647),
             ('outcomes', 622),
             ('teaching', 597),
             ('basic', 594),
             ('concepts', 586),
             ('prerequisites', 583),
             ('project', 581),
             ('keywords', 573),
             ('data', 572),
             ('work', 527),
             ('exercises', 527),
             ('models', 516),
             ('skills', 498),
             ('introduction', 487),
             ('activities', 476),
             ('theory', 461),
             ('lectures', 457),
             ('1', 444),
             ('2', 430),
             ('energy', 427),
             ('expected', 426),
             ('exam', 422),
         

I noticed that we have nasty concatenated words like communication9 or integratingspecific.
Let's try to do sth about it.

First issue can be easily solved with splitting on diigt.

In [13]:
def on_digit_split(s):
    return list(filter(None, re.split(r'(\d+)', s)))

In [14]:
on_digit_split('communication9')

['communication', '9']

Second issue is a little bit more elaborate than that and it solving would require some tricky dynamic programming apporach based on probability of word occuring in the dict. We are going to naively assume that such cases are rare and almost never happen, thus should have almost little or none influence for our tf-df. 

In [15]:
# Dict for word frequency count. (should it be per corpus or per document?)
words_used_freq = {}
processed_courses = unique_courses.copy()

for c_id, course in enumerate(unique_courses):
    description = course['description']
    
    list_of_words = []
    for i in wordpunct_tokenize(description):
        i_low = i.lower() #convert to lowercase
        split_on_digit = on_digit_split(i_low) #handle first issue where word is cocatenated with numbers.
        for w in split_on_digit:
            if (w not in stop_words) and w.isalpha(): #remove stopwords & punctuation & digit 'words'
                list_of_words += [w]
                if w in words_used_freq:
                    words_used_freq[w] += 1
                else:
                    words_used_freq[w] = 1
    processed_courses[c_id]['tokens'] = list_of_words

In [16]:
words_used_freq_ordered = OrderedDict(sorted(words_used_freq.items(),key = operator.itemgetter(1),reverse = True))
words_used_freq_ordered

OrderedDict([('methods', 1629),
             ('learning', 1478),
             ('student', 1213),
             ('content', 908),
             ('students', 840),
             ('design', 794),
             ('courses', 760),
             ('systems', 753),
             ('analysis', 722),
             ('end', 675),
             ('assessment', 647),
             ('outcomes', 622),
             ('teaching', 597),
             ('basic', 594),
             ('concepts', 587),
             ('prerequisites', 583),
             ('project', 582),
             ('keywords', 573),
             ('data', 573),
             ('work', 529),
             ('exercises', 527),
             ('models', 521),
             ('skills', 498),
             ('introduction', 490),
             ('activities', 476),
             ('theory', 465),
             ('lectures', 459),
             ('energy', 428),
             ('expected', 426),
             ('exam', 425),
             ('evaluate', 403),
             ('techniques',

### START:  TO BE DEFINED IF WE WANT IT OR NOT

In [17]:
EPFL_words = list(words_used_freq_ordered.keys())

In [18]:
# http://stackoverflow.com/questions/8870261/how-to-split-text-without-spaces-into-list-of-words

from math import log

# Build a cost dictionary, assuming Zipf's law and cost = -math.log(probability).
# http://stackoverflow.com/questions/772922/free-word-list-for-use-programmatically
# "/usr/share/dict/words" is not really working that well...

words = EPFL_words #open("/usr/share/dict/words").read().split()
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))
maxword = max(len(x) for x in words)

def infer_spaces(s):
    """Uses dynamic programming to infer the location of spaces in a string
    without spaces."""

    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters.
    # Returns a pair (match_cost, match_length).
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return " ".join(reversed(out))

In [19]:
#test
s = 'thumbgreenappleactiveassignmentweeklymetaphor'
print(infer_spaces(s))

thumb green apple active assignment weekly meta ph o r


In [20]:
print(infer_spaces('integratingspecific'))

integratingspecific


### END

Stem the words

In [21]:
stemmer=PorterStemmer()
all_stems = []

for c_id, course in enumerate(processed_courses):
    tokens = course['tokens']
    
    list_of_stems = []
    for token in tokens:
        list_of_stems.append(stemmer.stem(token))
    processed_courses[c_id]['stems'] = list_of_stems
    all_stems += list_of_stems

1. Explain which ones you implemented and why.

2. Print the terms in the pre-processed description of the IX class in alphabetical order.

In [22]:
processed_IX = next((item for item in processed_courses if item["courseId"] == "COM-308"))

In [23]:
processed_IX['stems'].sort()
processed_IX['stems']

['acquir',
 'activ',
 'ad',
 'ad',
 'advertis',
 'algebra',
 'algebra',
 'algorithm',
 'algorithm',
 'analysi',
 'analyt',
 'analyt',
 'apach',
 'applic',
 'applic',
 'assess',
 'auction',
 'auction',
 'balanc',
 'base',
 'base',
 'basic',
 'basic',
 'basic',
 'cathedra',
 'chain',
 'class',
 'class',
 'class',
 'cloud',
 'cluster',
 'cluster',
 'collect',
 'combin',
 'commerc',
 'commerc',
 'commun',
 'commun',
 'commun',
 'comput',
 'comput',
 'concept',
 'concept',
 'concret',
 'contain',
 'content',
 'cours',
 'cours',
 'coverag',
 'curat',
 'current',
 'data',
 'data',
 'data',
 'data',
 'data',
 'data',
 'dataset',
 'dataset',
 'decad',
 'dedic',
 'design',
 'detect',
 'detect',
 'dimension',
 'draw',
 'effect',
 'effici',
 'end',
 'exam',
 'expect',
 'explor',
 'explor',
 'explor',
 'explor',
 'explor',
 'field',
 'final',
 'foundat',
 'framework',
 'function',
 'fundament',
 'good',
 'graph',
 'graph',
 'hadoop',
 'hadoop',
 'hand',
 'homework',
 'homework',
 'import',
 'inform

In [24]:
all_stems_unique = list(set(all_stems))
all_stems_unique.sort()
all_stems_unique

['aa',
 'aand',
 'aarn',
 'ab',
 'abalo',
 'abat',
 'abb',
 'abc',
 'abcd',
 'abd',
 'abelian',
 'aberr',
 'abil',
 'abilitiy',
 'abilti',
 'abinitio',
 'ablat',
 'abml',
 'abnorm',
 'abou',
 'abroad',
 'absenc',
 'absolut',
 'absorb',
 'absorpt',
 'abstract',
 'abstractionpres',
 'abstrat',
 'abund',
 'abus',
 'ac',
 'academ',
 'academi',
 'academia',
 'academiab',
 'acceler',
 'acceleratorsacceler',
 'acceleratorscircular',
 'acceleratorsus',
 'acceleromet',
 'accept',
 'access',
 'accessori',
 'accid',
 'accidentsynthes',
 'acclaim',
 'accommod',
 'accompani',
 'accomplish',
 'accord',
 'account',
 'accountcomput',
 'accountpredict',
 'accredit',
 'accumul',
 'accur',
 'accuraci',
 'acheiv',
 'achiev',
 'acid',
 'acidif',
 'acidsdescrib',
 'ackerman',
 'acoust',
 'acoustc',
 'acousto',
 'acquaint',
 'acqui',
 'acquir',
 'acquisit',
 'acqusit',
 'act',
 'actin',
 'actinid',
 'action',
 'actiondesign',
 'actionmanipul',
 'actionoptim',
 'actionrecal',
 'actionsynthes',
 'activ',
 'act

## Exercise 4.2: Term-document matrix

In [25]:
# n x m matrix where n is number of terms and m is number of documents
n = len(all_stems_unique)
m = len(unique_courses)
TF_matrix = np.zeros((n, m))

In [26]:
n_indices = all_stems_unique
m_indices = [d['courseId'] for d in unique_courses]

In [27]:
TF_matrix.shape

(10875, 854)

In [28]:
# term_frequency
for c_id, course in enumerate(processed_courses):
    terms = course['stems']
    words_w_counts = Counter(terms)
    sum_of_terms = len(terms)
    for key, value in words_w_counts.items():
        n_id = n_indices.index(key)
        TF_matrix[n_id][c_id] = value

In [29]:
TF_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [30]:
# inverse document frequency
sum_of_documents = m
denom = np.apply_along_axis(np.count_nonzero, 1, TF_matrix) # doesnt work for some reason np.count_nonzero(TF_matrix, axis=1)
IDF_vector = np.log(sum_of_documents / denom)

In [31]:
IDF_vector

array([ 6.74993119,  6.74993119,  6.74993119, ...,  6.74993119,
        6.74993119,  6.05678401])

In [32]:
TFIDF_matrix = TF_matrix * np.array([IDF_vector]).T

In [33]:
TFIDF_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

1. Print the 15 terms in the description of the IX class with the highest TF-IDF scores. 

2. Explain where the difference between the large scores and the small ones comes from.

In [34]:
IX_index = m_indices.index('COM-308')

In [35]:
IX_TFIDF_col = TFIDF_matrix[:,IX_index]

In [36]:
IX_TFIDF_vals = {}
for ind, val in enumerate(IX_TFIDF_col):
    if not val > 0.0:
        continue
    IX_TFIDF_vals[n_indices[ind]] = val

In [37]:
IX_TFIDF_vals_ordered = OrderedDict(sorted(IX_TFIDF_vals.items(),key = operator.itemgetter(1),reverse = True))
IX_TFIDF_vals_ordered

OrderedDict([('mine', 18.681958608434936),
             ('onlin', 17.459173278835436),
             ('social', 15.695066405721727),
             ('explor', 15.061307877526007),
             ('world', 14.393650914403395),
             ('hadoop', 12.11356802645725),
             ('real', 11.42011537566993),
             ('servic', 10.843310933578261),
             ('auction', 10.727273665337359),
             ('commerc', 10.727273665337359),
             ('internet', 9.6080420894665135),
             ('retriev', 9.6080420894665135),
             ('network', 9.3728477860972674),
             ('dataset', 8.5300490880011388),
             ('stream', 8.369963672654066),
             ('data', 8.0297108516011804),
             ('ad', 7.9546849430975772),
             ('larg', 7.8683904262304338),
             ('cluster', 7.4108175121302935),
             ('graph', 7.3177774808605083),
             ('scale', 7.2575935605067166),
             ('lab', 7.1796671012969338),
             ('apach', 6

## Exercise 4.3: Document similarity search

In [38]:
'markov' in n_indices

True

In [39]:
'chain' in n_indices

True

In [40]:
markov_chain = np.zeros(n)

In [41]:
markov_chain[n_indices.index('markov')] = 1/2
markov_chain[n_indices.index('chain')] = 1/2

In [42]:
'facebook' in n_indices

True

In [43]:
facebook = np.zeros(n)
facebook[n_indices.index('facebook')] = 1

In [44]:
def cosine_sim(d1, d2):
    return np.dot(d1, d2) / (np.linalg.norm(d1) * np.linalg.norm(d2))

In [45]:
markov_chain_docs = np.apply_along_axis(cosine_sim, 0, TFIDF_matrix, markov_chain)

In [46]:
top_5_markov = np.argsort(markov_chain_docs)[-5:]

In [47]:
top_markov = {}
for top in top_5_markov:
    top_markov[unique_courses[top]['name']] = markov_chain_docs[top]

In [48]:
top_markov

{'Applied probability & stochastic processes': 0.55400769353626178,
 'Applied stochastic processes': 0.55211833344995109,
 'Markov chains and algorithmic applications': 0.38168653789318996,
 'Mathematical models in supply chain management': 0.31162506776787763,
 'Supply chain management': 0.37852761365218429}

In [49]:
facebook_docs = np.apply_along_axis(cosine_sim, 0, TFIDF_matrix, facebook)

In [50]:
top_5_facebook = np.argsort(facebook_docs)[-5:]

In [51]:
top_facebook = {}
for top in top_5_facebook:
    top_facebook[unique_courses[top]['name']] = facebook_docs[top]

In [52]:
top_facebook

{'CCMX Advanced Course - Instrumented Nanoindentation': 0.0,
 'Computational Social Media': 0.17945984867925696,
 'Electronic properties of solids and superconductivity': 0.0,
 'Hydrogeophysics': 0.0,
 'Molecular and cellular biophysic II': 0.0}